In [30]:
import numpy as np

In [31]:
# [0:5] regular with period about 60, 
# [5:14] constant acceleration of about 12, 
# [14:20] constant deceleration of about 8,
# [20:24] regular with period about 30

distances = np.array([0, 60.3, 121.1, 180.9, 241.9, 301.0, 307.3, 331.3, 366.9, 417.0, 478.7, 550.8, 632.6, 726.5, 831.6, 928.7, 1018.0, 1099.1, 1172.3, 1237.5, 1294.7, 1325.0, 1354.6, 1384.5, 1414.9])

In [32]:
n = len(distances)

velocities = np.array([y - x for (x, y) in zip(distances, distances[1:])])
accelerations = np.array([y - x for (x, y) in zip(velocities, velocities[1:])])

In [39]:
def r(j, k):
    m = np.median(velocities[j:k])
    return m, np.linalg.norm(velocities[j:k] - m, ord=1)

def a(j, k):
    m = np.median(accelerations[j:k-1])
    return m, np.linalg.norm(accelerations[j:k-1] - m, ord=1)

def fit_segment(j, k): # fits the segment from j to k and returns the type, error, and value
    (r_val, r_err), (a_val, a_err) = r(j, k), a(j, k)
    if a_err < r_err:
        return 'accelerating', a_err, a_val
    else:
        return 'regular', r_err, r_val

In [40]:
λ = 10 # hyperparameter

D = np.zeros(n)
partitions = [[0], [0, 1]]
partition_segments = [[], []]

for i in range(2, n):
    min_k = 0
    segment = fit_segment(0, i)
    min_cost = segment[1] + λ
    for k in range(2, i - 1):
        seg = fit_segment(k, i)
        cost = D[k] + seg[1] + λ
        if cost < min_cost:
            min_cost, min_k, segment = cost, k, seg
    D[i] = min_cost
    partitions.append(partitions[min_k] + [i])
    partition_segments.append(partition_segments[min_k] + [segment])

best_partition = zip(zip(partitions[-1], partitions[-1][1:]), partition_segments[-1])
best_partition = [dict(start=intvl[0], end=intvl[1], type=seg[0], error=seg[1], value=seg[2])
                  for (intvl, seg) in best_partition]

best_partition

[{'start': 0,
  'end': 6,
  'type': 'accelerating',
  'error': -3045.599999999998,
  'value': -0.9999999999999858},
 {'start': 6,
  'end': 8,
  'type': 'accelerating',
  'error': 0.0,
  'value': 11.599999999999966},
 {'start': 8,
  'end': 11,
  'type': 'regular',
  'error': 21.999999999999943,
  'value': 61.69999999999999},
 {'start': 11,
  'end': 24,
  'type': 'accelerating',
  'error': -4245.639999999999,
  'value': -7.849999999999966}]

In [29]:
# import matplotlib.pyplot as plt

# %matplotlib inline

# val = 0. # y-axis
# plt.plot(timestamps, np.zeros_like(timestamps) + val, 'x')

# plt.axvline(x=timestamps[partitions[n-1][0].end_index])
# plt.axvline(x=timestamps[partitions[n-1][1].end_index])
# plt.axvline(x=timestamps[partitions[n-1][2].end_index])
# plt.axvline(x=timestamps[partitions[n-1][3].end_index])

# first_loc = 0.75*timestamps[partitions[n-1][0].start_index]+0.25*timestamps[partitions[n-1][0].end_index]
# second_loc = 0.75*timestamps[partitions[n-1][1].start_index]+0.25*timestamps[partitions[n-1][1].end_index]
# third_loc = 0.75*timestamps[partitions[n-1][2].start_index]+0.25*timestamps[partitions[n-1][2].end_index]
# fourth_loc = 0.75*timestamps[partitions[n-1][3].start_index]+0.25*timestamps[partitions[n-1][3].end_index]

# first_text = partitions[n-1][0].kind + ": " + str(partitions[n-1][0].value)
# second_text = partitions[n-1][1].kind + ": " + str(partitions[n-1][1].value)
# third_text = partitions[n-1][2].kind + ": " + str(partitions[n-1][2].value)
# fourth_text = partitions[n-1][3].kind + ": " + str(partitions[n-1][3].value)

# plt.text(first_loc, 0.02, first_text, bbox=dict(facecolor='red', alpha=0.5))
# plt.text(second_loc, 0.02, second_text, bbox=dict(facecolor='red', alpha=0.5))
# plt.text(third_loc, 0.02, third_text, bbox=dict(facecolor='red', alpha=0.5))
# plt.text(fourth_loc, 0.02, fourth_text, bbox=dict(facecolor='red', alpha=0.5))